In [1]:
import pandas as pd
import numpy as np
import mysql.connector as mysql

db = mysql.connect(
    host = "localhost",
    user = "editekk",
    passwd = "editekk",
    database = "RidgeMeasurements"
)
cursor = db.cursor()

In [ ]:
cursor.execute("DROP TABLE pedicle_screw_averages")

In [2]:
exec = str(
    "CREATE TABLE pedicle_screw_averages " 
    # "(id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, "
    "(sample_id VARCHAR(15), "
    "fluence FLOAT(24), " 
    "dims VARCHAR(15), " 
    "batch VARCHAR(15), " 
    "image VARCHAR(50), "
    "radial_loc VARCHAR(3), "
    "linear_loc VARCHAR(7), "
    "thread_loc VARCHAR(7), "
    "wavlen_avg_validate FLOAT(10))")
    # "wavelen_avg_new FLOAT(10))")
cursor.execute(exec)

In [3]:
cursor.execute("SELECT sample_id, radial_loc, linear_loc, thread_loc FROM pedicle_screw_summary")
loc_data = cursor.fetchall()

cursor.execute("SELECT wavelength FROM pedicle_screw_summary")
wavlen_data = cursor.fetchall()

In [4]:
def get_vals_str(i):
    # TODO: CHECKME:
    return str(
        "SELECT line_count, "
        "sample_id, "
        "fluence, "
        "dims, "
        "batch, "
        "image, "
        "radial_loc, "
        "linear_loc, "
        "thread_loc "
        "FROM pedicle_screw_summary "
        "WHERE id=" + str(i+1))

def insert_vals(fet, avg_valid):
    _fetch = fet[0][1:] + tuple([avg_valid])
    exec = str(
            "INSERT INTO pedicle_screw_averages "
            " VALUES " + str(_fetch)
    )
    cursor.execute(exec)
    db.commit()


In [5]:
sum = wavlen_data[0][0]
line_count = 1
len_loc_data = len(loc_data)
for i in range(len_loc_data-1):
    if loc_data[i] == loc_data[i+1]:
        sum += wavlen_data[i+1][0]
        line_count += 1
    else:
        cursor.execute(get_vals_str(i))
        fetch = cursor.fetchall()
        avg = sum / line_count
        insert_vals(fetch, avg)
        sum = wavlen_data[i+1][0]
        line_count = 1

cursor.execute("SELECT line_count FROM pedicle_screw_summary ORDER BY ID DESC LIMIT 1")
last_count = cursor.fetchall()
cursor.execute(get_vals_str(len_loc_data-1))
last_row = cursor.fetchall()
avg = sum / last_count[0][0]
insert_vals(last_row, avg)